## Konwersja modelu do formatu TensorFlow.js

To ostatni krok w którym trzeba będzie korzystać z Pythona. Po konwersji modelu do formatu TensorFlow.js, będziemy mogli go wykorzystać w przeglądarce.

In [ ]:
from keras import models, layers, datasets, losses
import tensorflow as tf
import tensorflowjs as tfjs
import tempfile


# ------------------- Przykładowy model -------------------

model = models.Sequential(
    [
        layers.Input(shape=(32, 32, 3)),
        layers.Conv2D(64, 3, padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.MaxPooling2D(),
        layers.Conv2D(256, 3, padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.GlobalAveragePooling2D(),
        layers.Dense(256),
        layers.ReLU(),
        layers.Dense(10),
    ]
)
model.summary()

(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model.compile(
    optimizer="adam",
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(x_train, y_train, epochs=1, steps_per_epoch=1, batch_size=32)


# ------------------- Konwersja modelu do formatu TFJS -------------------

# Ścieżka do zapisu modelu w formacie TFJS
tfjs_save_path = "classifier_model.tfjs"
# Maksymalny rozmiar częsci wag modelu w bajtach
weight_shard_size_bytes = 1024 * 1024

# Zapis modelu do formatu TFJS
# Tworzenie tymczasowego katalogu dla modelu TF
with tempfile.TemporaryDirectory(suffix=".tf") as tf_path:
    # Zapis modelu w formacie TF
    tf.saved_model.save(model, tf_path)

    # Konwersja modelu do formatu TFJS
    tfjs.converters.convert_tf_saved_model(
        saved_model_dir=tf_path,
        output_dir=tfjs_save_path,
        weight_shard_size_bytes=weight_shard_size_bytes,
    )


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 440,970 (1.68 MB)

 Trainable params: 440,074 (1.68 MB)

 Non-trainable params: 896 (3.50 KB)

2024-12-05 22:44:27.273855: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1412', 24 bytes spill stores, 28 bytes spill loads

2024-12-05 22:44:27.409746: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1412', 24 bytes spill stores, 28 bytes spill loads

2024-12-05 22:44:27.577920: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1412', 24 bytes spill stores, 24 bytes spill loads



1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.0938 - loss: 2.3222


I0000 00:00:1733435069.483823    6241 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


INFO:tensorflow:Assets written to: /tmp/tmp9odagb_1.tf/assets


INFO:tensorflow:Assets written to: /tmp/tmp9odagb_1.tf/assets
I0000 00:00:1733435070.135018    6132 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1733435070.135133    6132 single_machine.cc:361] Starting new session
I0000 00:00:1733435070.135477    6132 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21769 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:06:00.0, compute capability: 8.6
